This is a demo of how to use 'rok_estimator.py'.

In [1]:
from rok_estimator import *
from lattice_lib import *
from lattice_lib.util import *

In [2]:
# Specify parameters related to the underlying ring
ring_params_60_32 = RingParam(f=60,nsis=2)

# Create a new starting relation
rel = Relation(ring_params=ring_params_60_32,wdim=60)
rel.show()

Relation:
    H * F * W = Y
Statement:
    H: nout x (ntop + nbot)
    F: (ntop + nbot) x wdim
    Y: nout x rep
Witness:
    W: wdim x rep
    ||sigma(W)||_2 <= 2^log_beta_wit_2
    ||psi(W)||_inf <= 2^log_beta_wit_inf
Parameters:
    wdim = 60, rep = 1, log_2-norm (real | extr) = (0  | 0 ), log_inf-norm (real | extr) = (0 | 0)
 


/home/russell/Nextcloud/Academic/Papers/rok-paper-sissors-estimator/rok_estimator.py:113: UserWarning: Specified module rank for SIS is too small for the target security level.
  warnings.warn("Specified module rank for SIS is too small for the target security level.")


In [3]:
# Running pi_bdecomp reducing rel to rel_bdecomp
rel_bdecomp, cost_bdecomp = rel.pi_bdecomp(ell=2)
rel_bdecomp.show()
cost_bdecomp.show()

Relation:
    H * F * W = Y
Statement:
    H: nout x (ntop + nbot)
    F: (ntop + nbot) x wdim
    Y: nout x rep
Witness:
    W: wdim x rep
    ||sigma(W)||_2 <= 2^log_beta_wit_2
    ||psi(W)||_inf <= 2^log_beta_wit_inf
Parameters:
    wdim = 60, rep = 2, log_2-norm (real | extr) = (8  | 0 ), log_inf-norm (real | extr) = (0 | 0)
 
Cost: communication =   0.10 KB, soundness error = 2^-Infinity
 


In [4]:
# Running pi_split reducing rel into rel_split
rel_split, cost_split = rel.pi_split(2)
rel_split.show()
cost_split.show()

Relation:
    H * F * W = Y
Statement:
    H: nout x (ntop + nbot)
    F: (ntop + nbot) x wdim
    Y: nout x rep
Witness:
    W: wdim x rep
    ||sigma(W)||_2 <= 2^log_beta_wit_2
    ||psi(W)||_inf <= 2^log_beta_wit_inf
Parameters:
    wdim = 30, rep = 2, log_2-norm (real | extr) = (0  | 0 ), log_inf-norm (real | extr) = (0 | 0)
 
Cost: communication =   0.10 KB, soundness error = 2^-128
 


In [5]:
# Running pi_batch reducing rel into rel_batch
rel_batch, cost_batch = rel.pi_batch()
rel_batch.show()
cost_batch.show()

Relation:
    H * F * W = Y
Statement:
    H: nout x (ntop + nbot)
    F: (ntop + nbot) x wdim
    Y: nout x rep
Witness:
    W: wdim x rep
    ||sigma(W)||_2 <= 2^log_beta_wit_2
    ||psi(W)||_inf <= 2^log_beta_wit_inf
Parameters:
    wdim = 60, rep = 1, log_2-norm (real | extr) = (0  | 0 ), log_inf-norm (real | extr) = (0 | 0)
 
Cost: communication =   0.00 KB, soundness error = 2^-128
 


In [6]:
# Running pi_fold reducing rel into rel_fold
rel_fold, cost_fold = rel.pi_fold(23)
rel_fold.show()
cost_fold.show()

Relation:
    H * F * W = Y
Statement:
    H: nout x (ntop + nbot)
    F: (ntop + nbot) x wdim
    Y: nout x rep
Witness:
    W: wdim x rep
    ||sigma(W)||_2 <= 2^log_beta_wit_2
    ||psi(W)||_inf <= 2^log_beta_wit_inf
Parameters:
    wdim = 60, rep = 23, log_2-norm (real | extr) = (3  | 0 ), log_inf-norm (real | extr) = (8 | 0)
 
Cost: communication =   0.00 KB, soundness error = 2^-83
 


In [7]:
# Running pi_norm reducing rel into rel_norm
rel_norm, cost_norm = rel.pi_norm()
rel_norm.show()
cost_norm.show()


Relation:
    H * F * W = Y
Statement:
    H: nout x (ntop + nbot)
    F: (ntop + nbot) x wdim
    Y: nout x rep
Witness:
    W: wdim x rep
    ||sigma(W)||_2 <= 2^log_beta_wit_2
    ||psi(W)||_inf <= 2^log_beta_wit_inf
Parameters:
    wdim = 60, rep = 8, log_2-norm (real | extr) = (7  | 0 ), log_inf-norm (real | extr) = (0 | 0)
 
Cost: communication =   5.60 KB, soundness error = 2^-122
 


In [8]:
ring_params = RingParam(f=60,log_betasis=32,log_q=64)
sim = Simulation(ring_params=ring_params)

rep = 2**5
wdim = 2**15
log_beta_wit_inf = 0
log_beta_wit_2 = ceil(log(sqrt(wdim * ring_params.phi * ring_params.fhat) * 2**log_beta_wit_inf,2))

rel = Relation(ring_params=ring_params,wdim=wdim,rep=rep,log_beta_wit_inf=log_beta_wit_inf,log_beta_wit_2=log_beta_wit_2)

ell = 2
d = 4

opener = [("norm", {}), ("batch", {}), ("split", {"d":d}), ("fold", {"repout":rep})]
loop = [("bdecomp", {"ell":ell}), ("norm", {}), ("batch", {}), ("split", {"d":d}), ("fold", {"repout":rep})]
ops = opener + loop + loop + opener + loop + [("finish", {})]

sim.simulate(rel, ops)
sim.show()

bdecomp: ell-inf norm is overestimated!
split: ell-inf norm is overestimated!
fold: ell-inf norm is overestimated!
split: ell-inf norm is overestimated!
bdecomp: ell-inf norm is overestimated!
split: ell-inf norm is overestimated!
bdecomp: ell-inf norm is overestimated!
split: ell-inf norm is overestimated!
init: ell-inf norm is overestimated!
Ring parameters:
conductor f: 60, degree phi: 16, modulus q: 2^64, beta_sis: 2^32
 
Execution Trace:
init    : wdim =  32768, rep =  32, log_2-norm (real | extr) = ( 12  |  12 ), log_inf-norm (real | extr) = (  0 |  12)
norm    : wdim =  32768, rep =  44, log_2-norm (real | extr) = ( 13  |  38*), log_inf-norm (real | extr) = (  0 |  38)
batch   : wdim =  32768, rep =  44, log_2-norm (real | extr) = ( 13  |  38*), log_inf-norm (real | extr) = (  0 |  38)
split   : wdim =   8192, rep = 176, log_2-norm (real | extr) = ( 13  |  38*), log_inf-norm (real | extr) = (  0 |  38)
fold    : wdim =   8192, rep =  32, log_2-norm (real | extr) = ( 23  |  29 ),